<a href="https://colab.research.google.com/github/claytonturner358/final-project/blob/main/Ancient_Metagenomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Install Kraken2 and related tools

In [ ]:
!apt-get update
!apt-get install -y kraken2

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,159 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/mai

### 2. Download the database from Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create a directory for the database
!mkdir -p kraken_db

# Define the path to the database in Google Drive
GOOGLE_DRIVE_DB_PATH = '/content/drive/MyDrive/minikraken2_v2_8GB_201904.tgz'
LOCAL_DB_ARCHIVE = 'kraken_db/minikraken2_v2_8GB_201904.tgz'

# Copy the database from Google Drive to the local kraken_db directory
!cp "{GOOGLE_DRIVE_DB_PATH}" "{LOCAL_DB_ARCHIVE}"

# Extract the database
# The tar command extracts contents into 'kraken_db/'. If the archive has a top-level folder, it will be created inside kraken_db.
!tar -xzf "{LOCAL_DB_ARCHIVE}" -C kraken_db/

# Set the database path to the *actual* directory containing the .k2d files, as confirmed by 'ls -R kraken_db' output.
KRAKEN_DB_PATH = 'kraken_db/minikraken2_v2_8GB_201904_UPDATE'
print(f"Kraken2 database path set to: {KRAKEN_DB_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kraken2 database path set to: kraken_db/minikraken2_v2_8GB_201904_UPDATE


In [ ]:
# This section of code is to ensure the proper  files were included with the kraken_db.
# Some downloads are missing the "taxo.k2d" file and this serves as a means of checking for it.

print("--- Contents of kraken_db directory ---")
!ls -R kraken_db

--- Contents of kraken_db directory ---
kraken_db:
minikraken2_v2_8GB_201904.tgz  minikraken2_v2_8GB_201904_UPDATE

kraken_db/minikraken2_v2_8GB_201904_UPDATE:
database100mers.kmer_distrib  database200mers.kmer_distrib  opts.k2d
database150mers.kmer_distrib  hash.k2d			    taxo.k2d


 3. Upload your FASTA file

In [ ]:
import os
from google.colab import files

# Attempt to upload a FASTA file
uploaded = files.upload()

if uploaded:
  INPUT_FASTA_FILE = list(uploaded.keys())[0]
  print(f'User uploaded file "{INPUT_FASTA_FILE}" with length {len(uploaded[INPUT_FASTA_FILE])} bytes')
else:
  print("No file uploaded. Please upload a FASTA file to proceed.")
  INPUT_FASTA_FILE = '' # Set to empty to prevent running Kraken2 with an invalid path

print(f"Using '{INPUT_FASTA_FILE}' as the input FASTA file.")

# Display the first few lines of the input FASTA file to verify its content
if INPUT_FASTA_FILE and os.path.exists(INPUT_FASTA_FILE):
  print(f"\n--- First 5 lines of {INPUT_FASTA_FILE} ---")
  with open(INPUT_FASTA_FILE, 'r') as f:
    for i, line in enumerate(f):
      if i >= 5: break
      print(line.strip())
  print("---------------------------------------")
else:
  print(f"Error: Input FASTA file '{INPUT_FASTA_FILE}' does not exist or is empty.")

Saving sample.fasta to sample.fasta
User uploaded file "sample.fasta" with length 164677241 bytes
Using 'sample.fasta' as the input FASTA file.

--- First 5 lines of sample.fasta ---
>d2e02fb3-8468-4439-9bcc-f196690fe333 runid=1cb309cec56f3c53b076b3a498abf9a7f6c9dde5 read=69 ch=766 start_time=2024-04-13T11:34:07.258289-05:00 flow_cell_id=PAS39809 protocol_group_id=test001 sample_id=acrotest001 parent_read_id=d2e02fb3-8468-4439-9bcc-f196690fe333 basecall_model_version_id=dna_r10.4.1_e8.2_400bps_hac@v4.3.0
TGTTGACCTCGTTCAGTTACGTATTGTTCGCGTGGCGTCCGGTTCTTCAGGCACGTTCTACAAAGCGCGCAGTGTGGCGTCTGGCTCGAGCTGCCAAGCGGCCGCGCGCACCTGTACGAGCGGCACGTTCGGCGGAGATGCTTCGTACCTCTACGCGACCTGTGAGGTGAGCCAGTTCAAGCACGAATACCACGGACAGCACAGGTCCGACGACAGCAATACATC
>f9172080-499d-4eb4-bf2c-8804bb66286f runid=1cb309cec56f3c53b076b3a498abf9a7f6c9dde5 read=56 ch=787 start_time=2024-04-13T11:34:07.258289-05:00 flow_cell_id=PAS39809 protocol_group_id=test001 sample_id=acrotest001 parent_read_id=f9172080-499d-4eb4-bf2c-8804bb66286

### 4. Run Kraken2 on your FASTA file

This command will classify your sequences. The output file (`sample.kraken`) will contain the classification for each read, and the report file (`sample.report`) will summarize the classifications.

In [ ]:
# Ensure INPUT_FASTA_FILE is the one uploaded by the user from cell 1693540a
KRAKEN_OUTPUT_FILE = 'sample.kraken'
KRAKEN_REPORT_FILE = 'sample.report'

!kraken2 --db "{KRAKEN_DB_PATH}" \
         --threads 4 \
         --output "{KRAKEN_OUTPUT_FILE}" \
         --report "{KRAKEN_REPORT_FILE}" \
         "{INPUT_FASTA_FILE}"

print(f"Kraken2 classification complete. Output: {KRAKEN_OUTPUT_FILE}, Report: {KRAKEN_REPORT_FILE}")

Loading database information... done.
200757 sequences (98.04 Mbp) processed in 4.517s (2666.4 Kseq/m, 1302.19 Mbp/m).
  29364 sequences classified (14.63%)
  171393 sequences unclassified (85.37%)
Kraken2 classification complete. Output: sample.kraken, Report: sample.report


### 5. Display the Kraken report

You can view the generated report file directly.

In [ ]:
print(f"--- Contents of {KRAKEN_REPORT_FILE} ---")
!cat "{KRAKEN_REPORT_FILE}"

--- Contents of sample.report ---
 85.37	171393	171393	U	0	unclassified
 14.63	29364	2877	R	1	root
 12.81	25709	52	R1	131567	  cellular organisms
  9.45	18972	1361	D	2	    Bacteria
  4.98	9998	109	D1	1783272	      Terrabacteria group
  4.68	9387	118	P	201174	        Actinobacteria
  4.40	8832	1451	C	1760	          Actinobacteria
  1.25	2510	27	O	85009	            Propionibacteriales
  0.76	1523	133	F	85015	              Nocardioidaceae
  0.39	782	64	G	1839	                Nocardioides
  0.14	278	278	S	196162	                  Nocardioides sp. JS614
  0.08	170	0	S	450734	                  Nocardioides dokdonensis
  0.08	170	170	S1	1300347	                    Nocardioides dokdonensis FR1436
  0.08	159	159	S	110319	                  Nocardioides sp. CF8
  0.06	111	111	S	2045452	                  Nocardioides sp. 78
  0.10	209	0	G	2044	                Pimelobacter
  0.10	209	209	S	2045	                  Pimelobacter simplex
  0.07	145	10	G	2040	                Aeromicrobium
  0.02	38	38	S	

### 6. Format the report with column headers and a taxonomic key

In [ ]:
# Define the output file name
OUTPUT_FORMATTED_REPORT_FILE = 'formatted_report.txt'

# Key for taxonomic levels
taxonomic_level_key = [
    "U: unclassified",
    "R: root",
    "D: domain",
    "P: phylum",
    "C: class",
    "O: order",
    "F: family",
    "G: genus",
    "S: species",
    "other single letters may represent subspecies, strain, or other finer distinctions"
]

# Column headers for the report
column_headers = [
    "Percentage",
    "Reads_Assigned_To_Clade",
    "Reads_Assigned_Directly",
    "Taxonomic_Level",
    "Taxon_ID",
    "Taxon_Name"
]

print(f"Formatting Kraken report and saving to {OUTPUT_FORMATTED_REPORT_FILE}...")

with open(KRAKEN_REPORT_FILE, 'r') as infile, open(OUTPUT_FORMATTED_REPORT_FILE, 'w') as outfile:
    # Write the key explanation
    outfile.write("### Kraken Report Taxonomic Level Key ###\n")
    for entry in taxonomic_level_key:
        outfile.write(f"# {entry}\n")
    outfile.write("#####################################\n\n")

    # Write the column headers
    outfile.write("\t".join(column_headers) + "\n")

    # Copy the content of the original report
    for line in infile:
        # Original Kraken report lines are tab-separated, so we can just write them directly
        # after the header.
        outfile.write(line)

print(f"Formatted report saved to: {OUTPUT_FORMATTED_REPORT_FILE}")


Formatting Kraken report and saving to formatted_report.txt...
Formatted report saved to: formatted_report.txt


In [ ]:
# Display the first few lines of the formatted report to verify
print(f"--- First 35 lines of {OUTPUT_FORMATTED_REPORT_FILE} ---")
with open(OUTPUT_FORMATTED_REPORT_FILE, 'r') as f:
    for i, line in enumerate(f):
        if i >= 35: break
        print(line.strip())
print("-----------------------------------------------------")


--- First 35 lines of formatted_report.txt ---
### Kraken Report Taxonomic Level Key ###
# U: unclassified
# R: root
# D: domain
# P: phylum
# C: class
# O: order
# F: family
# G: genus
# S: species
# other single letters may represent subspecies, strain, or other finer distinctions
#####################################

Percentage	Reads_Assigned_To_Clade	Reads_Assigned_Directly	Taxonomic_Level	Taxon_ID	Taxon_Name
85.37	171393	171393	U	0	unclassified
14.63	29364	2877	R	1	root
12.81	25709	52	R1	131567	  cellular organisms
9.45	18972	1361	D	2	    Bacteria
4.98	9998	109	D1	1783272	      Terrabacteria group
4.68	9387	118	P	201174	        Actinobacteria
4.40	8832	1451	C	1760	          Actinobacteria
1.25	2510	27	O	85009	            Propionibacteriales
0.76	1523	133	F	85015	              Nocardioidaceae
0.39	782	64	G	1839	                Nocardioides
0.14	278	278	S	196162	                  Nocardioides sp. JS614
0.08	170	0	S	450734	                  Nocardioides dokdonensis
0.08	170	170	S1	1